In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from rnnconv import RnnConv

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [4]:
E_conv_channels = 64
E_rnn1_channels = 256
E_rnn2_channels = 512
E_rnn3_channels = 512

B_conv_channels = 32

D_conv1_channels = 512
D_rnn1_channels = 512
D_rnn2_channels = 512
D_rnn3_channels = 256
D_rnn4_channels = 128
D_conv2_channels = 3

model = tf.keras.Sequential()      

model.add(layers.Conv2D(filters=E_conv_channels, kernel_size=3, input_shape=(32, 32, 3), activation = "relu", strides = (2,2), padding='same'))
model.add(RnnConv(filters=E_rnn1_channels, kernel_size=3, strides=(2,2), index=1))
model.add(RnnConv(filters=E_rnn2_channels, kernel_size=3, strides=(2,2), index=2))
model.add(RnnConv(filters=E_rnn3_channels, kernel_size=3, strides=(2,2), index=3))

model.add(layers.Conv2D(filters=B_conv_channels, kernel_size=1, activation = "relu", strides = (1,1), padding='same'))

model.add(layers.Conv2D(filters=B_conv_channels, kernel_size=1, activation = "relu", strides = (1,1), padding='same'))
model.add(RnnConv(filters=D_rnn1_channels, kernel_size=2, strides=(1,1), index=4))
model.add(layers.Lambda(lambda x:tf.nn.depth_to_space(x,2)))
model.add(RnnConv(filters=D_rnn2_channels, kernel_size=3, strides=(1,1), index=4))
model.add(layers.Lambda(lambda x:tf.nn.depth_to_space(x,2)))
model.add(RnnConv(filters=D_rnn3_channels, kernel_size=3, strides=(1,1), index=4))
model.add(layers.Lambda(lambda x:tf.nn.depth_to_space(x,2)))
model.add(RnnConv(filters=D_rnn4_channels, kernel_size=3, strides=(1,1), index=4))
model.add(layers.Lambda(lambda x:tf.nn.depth_to_space(x,2)))
model.add(layers.Conv2D(filters=D_conv2_channels, kernel_size=3, activation = "sigmoid", strides = (1,1), padding='same'))

model.compile(optimizer='adam', loss='mse', metrics=['mse','mae'])
model.build()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 16, 16, 64)        1792      
_________________________________________________________________
rnn_conv_7 (RnnConv)         (None, 8, 8, 256)         180992    
_________________________________________________________________
rnn_conv_8 (RnnConv)         (None, 4, 4, 512)         1312256   
_________________________________________________________________
rnn_conv_9 (RnnConv)         (None, 2, 2, 512)         2491904   
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 2, 2, 32)          16416     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 2, 2, 32)          1056      
_________________________________________________________________
rnn_conv_10 (RnnConv)        (None, 2, 2, 512)        

In [5]:
train_images = tf.convert_to_tensor(train_images)
model.fit(train_images, train_images, batch_size=64, epochs=1)

ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /home/jupyter/rnnconv.py:86 call  *
        self.new_hidden = tf.multiply(self.out_gate, Dense(1, activation="tanh")(self.new_cell))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:982 __call__  **
        self._maybe_build(inputs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:2643 _maybe_build
        self.build(input_shapes)  # pylint:disable=not-callable
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/layers/core.py:1178 build
        trainable=True)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:614 add_weight
        caching_device=caching_device)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/training/tracking/base.py:750 _add_variable_with_custo

In [23]:
# class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
#                'dog', 'frog', 'horse', 'ship', 'truck']

# plt.figure(figsize=(10,10))
# for i in range(25):
#     plt.subplot(5,5,i+1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.grid(False)
#     plt.imshow(train_images[i], cmap=plt.cm.binary)
#     # The CIFAR labels happen to be arrays, 
#     # which is why you need the extra index
#     plt.xlabel(class_names[train_labels[i][0]])
# # plt.show()

In [24]:
## # E_conv_channels = 64
# E_rnn1_channels = 256
# E_rnn2_channels = 512
# E_rnn3_channels = 512

# model = tf.keras.Sequential()      

# model.add(layers.Conv2D(filters=E_conv_channels, kernel_size=3, input_shape=(32, 32, 3), activation = "relu", strides = (2,2), padding='same'))
# # model.add(layers.MaxPooling2D(pool_size=(2,2)))    

# model.add(layers.Conv2D(filters=E_rnn1_channels, kernel_size=3, strides = (2,2), activation = "relu", padding='same'))    
# # model.add(layers.MaxPooling2D(pool_size=(2,2)))    
# model.add(layers.Flatten())
# model.add(layers.Lambda(lambda x: tf.expand_dims(model.output, axis=-1)))
# model.add(layers.LSTM(1, activation="relu", return_sequences=True))

# model.add(layers.Reshape((8, 8, E_rnn1_channels)))
# model.add(layers.Conv2D(filters=E_rnn2_channels, kernel_size=3, strides = (2,2), activation = "relu", padding='same'))    
# # model.add(layers.MaxPooling2D(pool_size=(2,2)))    
# model.add(layers.Flatten())
# model.add(layers.Lambda(lambda x: tf.expand_dims(model.output, axis=-1)))
# model.add(layers.LSTM(1, activation="relu", return_sequences=True))

# model.add(layers.Reshape((4, 4, E_rnn2_channels)))
# model.add(layers.Conv2D(filters=E_rnn3_channels, kernel_size=3, strides = (2,2), activation = "relu", padding='same'))    
# # model.add(layers.MaxPooling2D(pool_size=(2,2)))    
# model.add(layers.Flatten())
# model.add(layers.Lambda(lambda x: tf.expand_dims(model.output, axis=-1)))
# model.add(layers.LSTM(1, activation="relu", return_sequences=True))
# # model.add(layers.Lambda(lambda x: tf.squeeze(model.output, axis=0)))
# model.add(layers.Reshape((2,2, E_rnn3_channels)))
# # # model.add(layers.Lambda(lambda x: tf.squeeze(model.output, axis=0)))



# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [ ]:
E_conv_channels = 64
E_rnn1_channels = 256
E_rnn2_channels = 512
E_rnn3_channels = 512

model = tf.keras.Sequential()      

model.add(layers.Conv2D(filters=E_conv_channels, kernel_size=3, input_shape=(32, 32, 3), activation = "relu", strides = (2,2), padding='same'))
# model.add(layers.MaxPooling2D(pool_size=(2,2)))    

# model.add(layers.Flatten())
model.add(layers.Reshape((16*16*E_conv_channels,)))
model.add(layers.Lambda(lambda x: tf.expand_dims(model.output, axis=-1)))
model.add(layers.LSTM(1, activation="relu", return_sequences=True, return_state=True))
# model.add(layers.Lambda(lambda x: tf.squeeze(model.output, axis=-1)))
# model.add(layers.Reshape((16, 16, E_conv_channels)))
# model.add(layers.Conv2D(filters=E_rnn1_channels, kernel_size=3, strides = (2,2), activation = "relu", padding='same'))    

# model.add(layers.Flatten())
# model.add(layers.Lambda(lambda x: tf.expand_dims(model.output, axis=-1)))
# model.add(layers.LSTM(1, activation="relu", return_sequences=True, return_state=True))
# model.add(layers.Reshape((8, 8, E_rnn1_channels)))
# model.add(layers.Conv2D(filters=E_rnn2_channels, kernel_size=3, strides = (2,2), activation = "relu", padding='same')) 

# model.add(layers.Flatten())
# model.add(layers.Lambda(lambda x: tf.expand_dims(model.output, axis=-1)))
# model.add(layers.LSTM(1, activation="relu", return_sequences=True, ))
# model.add(layers.Reshape((4, 4, E_rnn2_channels)))
# model.add(layers.Conv2D(filters=E_rnn3_channels, kernel_size=3, strides = (2,2), activation = "relu", padding='same'))   

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()